## Extraer metadatos de archivo csv usando LLAMA3.1

In [ ]:
# Se ha instalado Ollama en local y se ha descargado el modelo LLAMA3.1 --> https://ollama.com/ (>> ollama run llama3.1)
# !pip install langchain

Creamos una función que extrae los metadatos técnicos y funcionales de un texto usando un LLM (en este caso LLAMA3.1).

In [2]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate

def extract_metadata(document_content,user_template):
    llm = ChatOllama(model="llama3.1",temperature=0)
    
    prompt_template = ChatPromptTemplate(messages=[
        ("user",user_template)
    ]
    )
    prompt = prompt_template.invoke([{"texto":document_content}])

    response = llm.invoke(prompt)
    response_print = print(response.content)
    return response_print

Definimos 2 templates para probar el funcionamiento. En el primero de ellos se pide una extracción de los metadatos técnicos y funcionales del texto, como indicamos anteriormente. Por otro lado, el segundo template lo creamos para probar el funcionamiento de las consultas del dataset, para ver si es capaz de encontrar el cliente con el saldo mayor (por ejemplo).

In [61]:
user_template1 = """\
#Extraer metadatos técnicos y funcionales del siguiente documento.
#Documento:{texto}
#Respuesta estructurada en dos apartados:
    *Metadatos técnicos
    *Metadatos funcionales

"""
user_template2 = "¿Quién es el cliente con mayor saldo? Solamente indica el id_cliente y su información en formato de tabla, no añadas explicaciones. Texto:{texto}"

En primera instancia, usamos CSVLoader para cargar un archivo csv y convertir cada instancia en un documento. Esto no es lo que queremos realizar todavía, pero nos sirve para crear embeddings de cada documento y montar una base de datos de vectores para usarla con un sistema RAG.

In [4]:
# !pip install langchain_community

In [ ]:
from langchain_community.document_loaders.csv_loader import CSVLoader

loader = CSVLoader('datos_prueba.csv')
document = loader.load()
for doc in document[:1]: #Prueba del funcionamiento, se llama al modelo en cada iteración (ideal para crear embeddings de cada instancia del dataset)
    extract_metadata(doc.page_content,user_template1)

En esta primera fase, lo que queremos es extraer los metadatos técnicos y funcionales de un dataset. Para ello, utilizaremos la librería pandas, con la que haremos la ingesta del archivo (un csv de momento) y prepararemos la información necesaria para pasársela al LLM. Como primera aproximación, convertiremos el dataset en un string, que pasaremos al modelo, para ver el comportamiento del mismo. Más adelante, se podrían realizar transformaciones más estructuradas, como por ejemplo, convertir a json y posteriormente a string.
A modo de prueba, solamente tomaremos 25 instancias del dataset.

In [32]:
import pandas as pd
df = pd.read_csv('datos_prueba.csv')
df_25=df.head(25)
df_string = df_25.to_string()
print(df_string)

    id_cliente          nombre                 email       telefono        dni fecha_nacimiento    saldo
0         1001     María López      ana8@dominio.com  +34 620526944  14315522O       1977-07-28  5385.31
1         1002    Laura Torres   maría20@outlook.com  +34 610417643  16899804I       1961-11-21  2678.78
2         1003   Pedro Sánchez   pedro79@dominio.com  +34 610897848  98233516E       1997-09-13  4842.24
3         1004  Luis Fernández  sergio48@empresa.org  +34 610626959  18282590K       1961-07-26   783.21
4         1005     María López     pedro87@yahoo.com  +34 630193110  94877483O       1980-02-24  6778.78
5         1006   Elena Ramírez   marta49@outlook.com  +34 600144425  65994640K       2002-09-19   516.32
6         1007       Ana Gómez   marta34@dominio.com  +34 640955695  62142091L       1989-10-14  3724.59
7         1008       Ana Gómez   clara41@outlook.com  +34 630332580  55383009W       2004-05-16  7989.58
8         1009    Sergio Vidal     laura22@gmail.com  +

Visualizamos la respuesta del LLM al primer prompt. En este caso, podemos observar que nos da una estructura de los metadatos técnicos y funcionales del dataset bastante correcta. 

In [46]:
# Uso del primer template
extract_metadata(df_string,user_template1)

**Metadatos técnicos**

* **Tipo de datos**: La respuesta es un documento JSON que contiene una lista de diccionarios.
* **Estructura del documento**: Cada diccionario tiene las siguientes claves:
 + `id_cliente`: identificador único del cliente
 + `nombre`: nombre del cliente
 + `email`: dirección de correo electrónico del cliente
 + `telefono`: número de teléfono del cliente
 + `dni`: número de identificación del cliente (DNI)
 + `fecha_nacimiento`: fecha de nacimiento del cliente
 + `saldo`: saldo actual del cliente
* **Características técnicas**:
 + El documento tiene 25 filas y 8 columnas.
 + Los datos están almacenados en formato texto, con un separador de columnas no especificado (posiblemente sea tabulación o coma).
 + No hay información sobre la base de datos o el sistema de gestión de bases de datos utilizado.

**Metadatos funcionales**

* **Descripción del contenido**: El documento contiene una lista de clientes con sus respectivas características, incluyendo identificador ú

Ahora utilizamos el segundo prompt, para ver si el modelo es capaz de encontrar al cliente con el saldo mayor.

In [37]:
# Uso del segundo template
extract_metadata(df_string,user_template2)

| id_cliente | nombre         | email                | telefono          | dni      | fecha_nacimiento | saldo |
|------------|----------------|----------------------|--------------------|----------|------------------|-------|
| 1008       | Ana Gómez     | clara41@outlook.com  | +34 630332580    | 55383009| 2004-05-16      | 7989.58|


Comprobamos si es correcto lo que ha dicho el LLM. Podemos observar que se ha equivocado, ya que el id_cliente con mayor saldo es el 1015, no el 1008.

In [ ]:
df_sorted = df_25.sort_values(by='saldo', ascending=False)

cliente_con_mayor_saldo = df_sorted.iloc[0]

# Mostrar la información del cliente con mayor saldo
print(cliente_con_mayor_saldo)

id_cliente                       1015
nombre                   Laura Torres
email               ana24@dominio.com
telefono                +34 610565529
dni                         67103973X
fecha_nacimiento           1994-08-26
saldo                         9849.42
Name: 14, dtype: object


Como podemos observar en las iteraciones anteriores, si le pasamos al modelo 25 instancias del dataset (porque todo el dataset se demora mucho ya que son 1000 instancias) y le preguntamos por el cliente con mayor saldo nos da el id_cliente 1008, el cual si lo comparamos con la consulta con pandas, es incorrecto. El modelo no realiza operaciones numéricas reales; solo interpreta texto. Por eso, aunque vea los números en un JSON o CSV, no los compara con precisión como lo haría un programa. Solo aparenta razonar, pero no calcula. En estos casos, lo ideal sería crear un agente que tenga alguna herramienta relacionada con ejecución de código.


Recomiendo mirar la documentación oficial de LangChain porque han habido cambios en la nueva versión (langchain v0.3) --> https://python.langchain.com/docs/introduction/